In [0]:
# Imports
from IPython.display import Image
import pandas as pd
import numpy as np
from scipy.stats import binom_test

In [0]:
# READ CSV - Method 2 (Drive)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Read tables

In [0]:
pd.set_option("display.max_colwidth", 100)

### Patients table

In [0]:
mimic_patients_df = pd.read_csv(PATH + 'mimic-III/PATIENTS.csv')

### Admission table

In [0]:
mimic_admission_df = pd.read_csv(PATH + 'mimic-III/ADMISSIONS.csv')

#### Merge both tables and calculate true age

In [0]:
# Merge patient and admission tables
mimic_patients_admission_df = pd.merge(mimic_admission_df, mimic_patients_df, on = 'SUBJECT_ID' , how = 'inner')

In [0]:
# ADMITTIME and DOB objects to datetime
mimic_patients_admission_df["ADMITTIME"] = pd.to_datetime(mimic_patients_admission_df["ADMITTIME"], format='%Y%m%d %H:%M:%S')
mimic_patients_admission_df["DOB"] = pd.to_datetime(mimic_patients_admission_df["DOB"], format='%Y%m%d %H:%M:%S')

# True age
mimic_patients_admission_df["AGES"] = mimic_patients_admission_df["ADMITTIME"].sub(mimic_patients_admission_df["DOB"]).dt.days/365.242

# Adult patients only
mimic_patients_admission_df = mimic_patients_admission_df[(mimic_patients_admission_df.AGES >= 18)]

### ICD-9 Codes table

In [0]:
mimic_diagnoses_df = pd.read_csv(PATH + 'mimic-III/DIAGNOSES_ICD.csv')

In [0]:
mimic_patients_diagnoses_df = pd.merge(mimic_patients_admission_df, mimic_diagnoses_df, on = 'SUBJECT_ID' , how = 'inner')

### ICD-9 Descriptions table

In [0]:
mimic_diagnoses_descriptions_df = pd.read_csv(PATH + 'mimic-III/D_ICD_DIAGNOSES.csv')

## Occurrence and Mortality

### By insurance

In [0]:
insurance_mortality_df = mimic_patients_diagnoses_df.groupby(['ICD9_CODE', 'INSURANCE', 'HOSPITAL_EXPIRE_FLAG']).size().unstack()
# insurance_mortality_df = insurance_mortality_df.reset_index()
# insurance_mortality_df.columns.names = [None]
# insurance_mortality_df.columns = ['ICD9_CODE', 'RACE', 'ALIVE', 'DEAD']
# insurance_mortality_df.insert(4, 'TOTAL', 'NULL')


# # Compute alive, dead and total
# for index, row in insurance_mortality_df.iterrows():
#   insurance_mortality_df.at[index, 'TOTAL'] = row.ALIVE + row.DEAD
  

# insurance_mortality_df.groupby('RACE')['DEAD'].count()

In [0]:
insurance_mortality_df

HOSPITAL_EXPIRE_FLAG       0      1
ICD9_CODE INSURANCE                
0030      Medicare       4.0    NaN
0031      Medicare       1.0    NaN
0038      Medicare       1.0    NaN
          Private        1.0    NaN
0039      Private        1.0    NaN
0041      Medicare       1.0    NaN
          Private        2.0    NaN
0048      Private        1.0    NaN
0049      Government     2.0    NaN
0051      Medicare       3.0    NaN
00581     Medicare       1.0    1.0
          Private        1.0    NaN
0059      Medicaid       1.0    NaN
0071      Medicaid       NaN    1.0
0074      Medicaid       2.0    1.0
          Medicare       9.0    NaN
          Private        4.0    2.0
00804     Private        1.0    NaN
00841     Medicare       4.0    NaN
00843     Medicare       2.0    NaN
00845     Government    24.0    4.0
          Medicaid     218.0   17.0
          Medicare    2042.0  268.0
          Private      617.0   69.0
          Self Pay       6.0    NaN
00847     Private        2.0    1.0
0085      Medicare       8.0    NaN
          Private        3.0    NaN
00861     Private        1.0    NaN
00862     Medicare       1.0    NaN
...                      ...    ...
V860      Private       10.0    NaN
V861      Private        2.0    NaN
V8709     Private        NaN    1.0
V872      Medicare       2.0    NaN
          Private        4.0    NaN
          Self Pay       1.0    NaN
V8741     Government    12.0    2.0
          Medicaid      47.0    7.0
          Medicare     329.0   56.0
          Private      250.0   33.0
V8745     Medicare       1.0    NaN
V8801     Government     2.0    NaN
          Medicaid      36.0    NaN
          Medicare     183.0   11.0
          Private       53.0    1.0
          Self Pay       1.0    NaN
V8811     Private        1.0    NaN
V8812     Government     3.0    NaN
          Medicaid       3.0    NaN
          Medicare       4.0    NaN
          Private        5.0    NaN
V8821     Medicaid       2.0    NaN
          Medicare      10.0    NaN
V9010     Private        1.0    NaN
V902      Medicare       NaN    1.0
V9039     Private        1.0    NaN
V9081     Medicare       1.0    NaN
          Private        2.0    NaN
V9089     Medicaid       2.0    NaN
V9103     Private        1.0    NaN

[17814 rows x 2 columns]

In [0]:
# # 1. Mortality significance: only +2 patients dead
# race_mortality_df = race_mortality_df[race_mortality_df['DEAD'] > 2]


# # 2. Remove ICD9 codes with only ONE race
# for index, row in race_mortality_df.iterrows():
#   rows = race_mortality_df[race_mortality_df["ICD9_CODE"] == row.ICD9_CODE]
#   if (len(rows) == 1):
#     race_mortality_df.drop(rows.index, inplace=True)

In [0]:
# 3. Binomial test

# icd9_code = []
# gender = []
# p_value_occ = []
# p_value_mort = []
# alive_m = []
# dead_m = []
# total_m = []
# alive_f = []
# dead_f = []
# total_f = []

# icd_pass = False
 
# for index, row in race_mortality_df.iterrows():
#   if not icd_pass:
#     rows = race_mortality_df[race_mortality_df["ICD9_CODE"] == row.ICD9_CODE]
#     icd_pass = True
    
#     print(rows)
#     print(len(rows))
    
#     # If less common in white/black/asian/latino but they die more
#     if(((rows.loc[rows.RACE == "WHITE"]["TOTAL"] < rows.iloc[1]["TOTAL"]) and (rows.iloc[0]["DEAD"] > rows.iloc[1]["DEAD"])) or
#        ((rows.loc[1]["TOTAL"] < rows.iloc[0]["TOTAL"]) and (rows.iloc[1]["DEAD"] > rows.iloc[0]["DEAD"]))): 
#       
#       n_occ = rows.iloc[0]["TOTAL"] + rows.iloc[1]["TOTAL"]
      
#       alive_f.append(rows.iloc[0]["ALIVE"])
#       dead_f.append(rows.iloc[0]["DEAD"])
#       total_f.append(rows.iloc[0]["TOTAL"])
      
#       alive_m.append(rows.iloc[1]["ALIVE"])
#       dead_m.append(rows.iloc[1]["DEAD"])
#       total_m.append(rows.iloc[1]["TOTAL"])
      
      
#       if((rows.iloc[0]["TOTAL"] < rows.iloc[1]["TOTAL"]) and (rows.iloc[0]["DEAD"] > rows.iloc[1]["DEAD"])):
#         x_occ = rows.iloc[0]["TOTAL"] # Women
#         x_mort = rows.iloc[0]["DEAD"]
#         gender.append('F')
#         icd9_code.append(rows.iloc[0]['ICD9_CODE'])
#         prob_occ = rows.iloc[1]["TOTAL"]/n_occ
#         prob_mort = rows.iloc[1]["DEAD"]/rows.iloc[1]["TOTAL"]
#       else:
#         x_occ = rows.iloc[1]["TOTAL"] # Men
#         x_mort = rows.iloc[1]["DEAD"]
#         gender.append('M')
#         icd9_code.append(rows.iloc[1]['ICD9_CODE'])
#         prob_occ = rows.iloc[0]["TOTAL"]/n_occ
#         prob_mort = rows.iloc[0]["DEAD"]/rows.iloc[0]["TOTAL"]
         
#       # p-value: binom test
#       p_value_occ.append(binom_test(x = x_occ, n = n_occ, p = prob_occ, alternative = 'two-sided'))
#       p_value_mort.append(binom_test(x = x_mort, n = x_occ, p = prob_mort, alternative = 'two-sided'))
#   else:
#     icd_pass = False

In [0]:
# dict = {'Gender': gender, 'ICD9': icd9_code, 'Occ p-value': p_value_occ, 
#         'Mort p-value': p_value_mort, 'Alive F': alive_f, 'Dead F': dead_f, 
#         'Total F': total_f, 'Alive M': alive_m, 'Dead M': dead_m, 'Total M': total_m,}  
    
# binom_test_df = pd.DataFrame(dict)
# binom_test_df = binom_test_df.merge(mimic_diagnoses_descriptions_df, left_on='ICD9', right_on='ICD9_CODE')
# binom_test_df = binom_test_df.drop(['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE'], axis=1)

# # 5%
# binom_test_df[(binom_test_df['Occ p-value'] < 0.05) & (binom_test_df['Mort p-value'] < 0.05)]